In [ ]:
import requests # 2.31.0
import pandas as pd # 2.2.2

class AllMarketInfo:
    """Class to fetch and preprocess cryptocurrency market data from various exchanges."""

    def __init__(self):
        # Base URLs and additional paths for each exchange
        self.api_config = {
            "upbit": {
                "base_url": "https://api.upbit.com",
                "add_url": "/v1/market/all"
            },
            "bithumb": {
                "base_url": "https://api.bithumb.com",
                "add_url": "/public/ticker/ALL"
            },
            "coinone": {
                "base_url": "https://api.coinone.co.kr",
                "add_url": "/public/v2/currencies"
            },
            "korbit": {
                "base_url": "https://api.korbit.co.kr",
                "add_url": "/v2/currencyPairs"
            },
            "gopax": {
                "base_url": "https://api.gopax.co.kr",
                "add_url": "/assets"
            }
        }
        self.headers = {"accept": "application/json"}

    def call_response(self, exchange):
        """Fetch JSON response from the specified exchange."""
        try:
            url = self.api_config[exchange]["base_url"] + self.api_config[exchange]["add_url"]
            response = requests.get(url, headers=self.headers)
            
            if response.status_code == 200:
                return response.json()
            else:
                print(f"Error: {response.status_code} while fetching data from {exchange}")
                return None
        except KeyError:
            print(f"Exchange '{exchange}' is not configured.")
            return None

    def preprocess_data(self, exchange):
        """Preprocess data based on the exchange."""
        market_data = self.call_response(exchange)
        if market_data:
            if exchange == "upbit":
                return self.upbit_preprocessing(market_data)
            elif exchange == "bithumb":
                return self.bithumb_preprocessing(market_data)
            elif exchange == "coinone":
                return self.coinone_preprocessing(market_data)
            elif exchange == "korbit":
                return self.korbit_preprocessing(market_data)
            elif exchange == "gopax":
                return self.gopax_preprocessing(market_data)
        return None

    def upbit_preprocessing(self, market_data):
        """Preprocess Upbit data."""
        processed_data = [
            {
                "symbol": item["market"].split("-")[1].upper(),
                "korean_name": item["korean_name"],
                "english_name": item["english_name"]
            }
            for item in market_data
        ]
        return pd.DataFrame(processed_data)

    def bithumb_preprocessing(self, market_data):
        """Preprocess Bithumb data."""
        processed_data = [
            {
                "symbol": key.upper(),
                "korean_name": value.get("korean_name", ""),
                "english_name": value.get("english_name", "")
            }
            for key, value in market_data.items() if key != 'date'
        ]
        return pd.DataFrame(processed_data)

    def coinone_preprocessing(self, market_data):
        """Preprocess Coinone data."""
        processed_data = [
            {
                "symbol": item["symbol"].upper(),
                "english_name": item["name"]
            }
            for item in market_data["currencies"]
        ]
        return pd.DataFrame(processed_data)

    def korbit_preprocessing(self, market_data):
        """Preprocess Korbit data."""
        # Extract 'data' field from the response
        if "data" in market_data:
            processed_data = [
                {"symbol": item["symbol"].split("_")[0].upper()} for item in market_data["data"]
            ]
            return pd.DataFrame(processed_data)
        else:
            print("Error: 'data' field not found in Korbit response.")
            return pd.DataFrame()  # Return an empty DataFrame in case of error


    def gopax_preprocessing(self, market_data):
        """Preprocess Gopax data."""
        processed_data = [
            {
                "symbol": item["id"].upper(),
                "korean_name": item.get("name", ""),
                "english_name": item.get("englishName", "")
            }
            for item in market_data
        ]
        return pd.DataFrame(processed_data)

    def fetch_and_save(self, exchange, file_path=None):
        """
        Fetch data from the specified exchange, preprocess it,
        and save it as a CSV file if a file path is provided.
        
        Args:
          - exchange (str): The name of the exchange (e.g., 'upbit', 'bithumb').
          - file_path (str): Optional. File path to save the DataFrame as a CSV.
        
        Returns:
          - DataFrame: The preprocessed DataFrame.
          - Saves the DataFrame to a CSV file if file_path is provided.
        """
        df = self.preprocess_data(exchange)
        
        if df is not None:
            if file_path:
                df.to_csv(file_path, index=False)
                print(f"Data saved to {file_path}")
            
            return df
        else:
            print(f"Failed to fetch or preprocess data for {exchange}")
